In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.cuda.is_available()

True

In [97]:
class Net(nn.Module):

    def __init__(self):
        super(Net,self).__init__()
        "input size 32 * 32, input channel 1, output channel 6, 3*3 convolution"
        self.conv1 = nn.Conv2d(1,6,3) # in_channels, out_channels, kernal_size
        #output: 6 @ 30 * 30
        #2*2 max pool, output: 6 @ 15 * 15
        self.conv2 = nn.Conv2d(6,16,3)
        #output: 16 @ 13 * 13
        #2*2 max pool, output: 16 @ 6 * 6
        self.fc1   = nn.Linear(16 * 6 * 6, 120)
        self.fc2   = nn.Linear(120,84)
        self.fc3   = nn.Linear(84,10)

    def forward(self,x):
        x = F.max_pool2d(F.relu(self.conv1(x)),(2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        x = x.view(-1,self.num_flat_features(x))
        """
        view: share the same data, avoids copy, and have two dimentions: 
        num_batches * num_features. pass -1 means this dimension will 
        adjust automatically to the rest.
        """
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x
    
    def num_flat_features(self,x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s

        return num_features

In [98]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [99]:
input = torch.randn(1,1,32,32)#num_batches, in_channels, height, width
output = net(input)

In [7]:
input = torch.randn(20,4,10) #num_batches, in_channels, length
input.view(in)

torch.Size([20, 5, 8])

In [101]:
net.zero_grad()
output.backward(torch.randn(1, 10))
#TODO: WHY?

Loss function

In [106]:
target = torch.randn(1,10)
loss = nn.MSELoss()(output,target)
print(loss)


tensor(1.0146, grad_fn=<MseLossBackward>)


In [8]:
F.tanh

<function torch.nn.functional.tanh(input)>

In [10]:
class DQN(nn.Module):
    def __init__(self,env:TradingEnv):
        self.in_channels = env.observation_space.shape[0]  #back looking window. Could be 30?
        self.length = env.observation_space.shape[1]       #normally should be 6: open close high low holdings nav
        self.num_actions = env.action_space.n

        self.conv1 = nn.Conv1d(in_channels,32,3)      #should be num_batches * 32 * 4
        self.conv2 = nn.Conv1d(32,8,3)                #should be num_batches * 8 * 2
        self.fc    = nn.Linear(8 * (length - 4), env.action_space.n)

    def forward(self,x):
        x = F.tanh(self.conv1(x))
        x = F.tanh(self.conv2(x))
        x = x.view(-1,self.num_flat_features(x))
        x = self.fc(x)

        return x

    def num_flat_features(self,x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s

        return num_features

NameError: name 'TradingEnv' is not defined